In [ ]:
from py2neo import Graph,Node,Relationship

In [ ]:
import pandas as pd
import os
import QUANTAXIS as QA
import datetime
import numpy as np 
import statsmodels.formula.api as sml
from QAStrategy.qastockbase import QAStrategyStockBase

import matplotlib.pyplot as plt
import scipy.stats as scs
import matplotlib.mlab as mlab
from easyquant.indicator.base import *

import json
from easyquant import MongoIo
import statsmodels.api as sm
from multiprocessing import Process, Pool, cpu_count, Manager


In [ ]:
mongo = MongoIo()


In [ ]:
def tdx_base_func(data, code_list = None):
    """
    准备数据
    """
    # highs = data.high
    # start_t = datetime.datetime.now()
    # print("begin-tdx_base_func:", start_t)
    if len(data) < 10:
        data = data.copy()
        data['bflg'] = 0
        data['sflg'] = 0
        return data

    CLOSE=data.close
    C=data.close
    # df_macd = MACD(C,12,26,9)
    # mtj1 = IFAND(df_macd.DIFF < 0, df_macd.DEA < 0, 1, 0)
    # mtj2 = IFAND(mtj1, df_macd.MACD < 0, 1, 0)
    花 = SLOPE(EMA(C, 3), 3)
    神 = SLOPE(EMA(C, 7), 7)
    买 = IFAND(COUNT(花 < 神, 5)==4 , 花 >= 神,1,0)
    卖 = IFAND(COUNT(花 >= 神, 5)==4, 花 < 神,1,0)
    钻石 = IFAND(CROSS(花, 神), CLOSE / REF(CLOSE, 1) > 1.03, 1, 0)
    买股 = IFAND(买, 钻石,1,0)
    # 买股 = IFAND(mtj2, 买股1, 1, 0)
    # AND(CROSS(花, 神)
    # AND
    # CLOSE / REF(CLOSE, 1) > 1.03);

    # return pd.DataFrame({'FLG': 后炮}).iloc[-1]['FLG']
    # return 后炮.iloc[-1]

    # 斜率
    data = data.copy()
    # data['bflg'] = IF(REF(后炮,1) > 0, 1, 0)
    data['bflg'] = 买股
    data['sflg'] = 卖
    # print("code=%s, bflg=%s" % (code, data['bflg'].iloc[-1]))
    # data['beta'] = 0
    # data['R2'] = 0
    # beta_rsquared = np.zeros((len(data), 2),)
    #
    # for i in range(N - 1, len(highs) - 1):
    # #for i in range(len(highs))[N:]:
    #     df_ne = data.iloc[i - N + 1:i + 1, :]
    #     model = sml.ols(formula='high~low', data = df_ne)
    #     result = model.fit()
    #
    #     # beta = low
    #     beta_rsquared[i + 1, 0] = result.params[1]
    #     beta_rsquared[i + 1, 1] = result.rsquared
    #
    # data[['beta', 'R2']] = beta_rsquared

    # 日收益率
    data['ret'] = data.close.pct_change(1)

    # 标准分
    # data['beta_norm'] = (data['beta'] - data.beta.rolling(M).mean().shift(1)) / data.beta.rolling(M).std().shift(1)
    #
    # beta_norm = data.columns.get_loc('beta_norm')
    # beta = data.columns.get_loc('beta')
    # for i in range(min(M, len(highs))):
    #     data.iat[i, beta_norm] = (data.iat[i, beta] - data.iloc[:i - 1, beta].mean()) / data.iloc[:i - 1, beta].std() if (data.iloc[:i - 1, beta].std() != 0) else np.nan

    # data.iat[2, beta_norm] = 0
    # data['RSRS_R2'] = data.beta_norm * data.R2
    # data = data.fillna(0)
    #
    # # 右偏标准分
    # data['beta_right'] = data.RSRS_R2 * data.beta
    # if code == '000732':
    #     print(data.tail(22))

    return data


In [25]:
def buy_sell_fun(price, S1=1.0, S2=0.8):
    """
    斜率指标交易策略标准分策略
    """
    data = price.copy()
    data['flag'] = 0 # 买卖标记
    data['position'] = 0 # 持仓标记
    data['hold_price'] = 0  # 持仓价格
    bflag = data.columns.get_loc('bflg')
    sflag = data.columns.get_loc('sflg')
    # beta = data.columns.get_loc('beta')
    flag = data.columns.get_loc('flag')
    position_col = data.columns.get_loc('position')
    close_col = data.columns.get_loc('close')
    high_col = data.columns.get_loc('high')
    open_col = data.columns.get_loc('open')
    hold_price_col = data.columns.get_loc('hold_price')
    position = 0 # 是否持仓，持仓：1，不持仓：0
    for i in range(1,data.shape[0] - 1):
        # 开仓
        if data.iat[i, bflag] > 0 and position == 0:
            data.iat[i, flag] = 1
            data.iat[i, position_col] = 1
            data.iat[i, hold_price_col] = data.iat[i, open_col]
            data.iat[i + 1, position_col] = 1
            data.iat[i + 1, hold_price_col] = data.iat[i, open_col]

            position = 1

            print("buy  : date=%s code=%s price=%.2f" % (data.iloc[i].name[0], data.iloc[i].name[1], data.iloc[i].close))
            code = data.iloc[i].name[1]
            price = data.iloc[i].close
#             qa_order.send_order('BUY', 'OPEN', code=code, price=price, volume=1000)
            # order.send_order('SELL', 'CLOSE', code=code, price=price, volume=1000)
        # 平仓
        # elif data.iat[i, bflag] == S2 and position == 1:
        elif data.iat[i, position_col] > 0 and position == 1:
            cprice = data.iat[i, close_col]
            # oprice = data.iat[i, open_col]
            hole_price = data.iat[i, hold_price_col]
            high_price = data.iat[i, high_col]
            if cprice < hole_price * 0.95:# or cprice > hprice * 1.2:
                data.iat[i, flag] = -1
                data.iat[i + 1, position_col] = 0
                data.iat[i + 1, hold_price_col] = 0
                position = 0
                print("sell : code=%s date=%s  price=%.2f" % (data.iloc[i].name[0], data.iloc[i].name[1], data.iloc[i].close))
                code = data.iloc[i].name[1]
                price = data.iloc[i].close
                # order.send_order('BUY', 'OPEN', code=code, price=price, volume=1000)
#                 qa_order.send_order('SELL', 'CLOSE', code=code, price=price, volume=1000)

            elif cprice > hole_price * 1.1 and high_price / cprice > 1.05:
                data.iat[i, flag] = -1
                data.iat[i + 1, position_col] = 0
                data.iat[i + 1, hold_price_col] = 0
                position = 0
                print("sell : code=%s date=%s  price=%.2f" % (data.iloc[i].name[0], data.iloc[i].name[1], data.iloc[i].close))
                code = data.iloc[i].name[1]
                price = data.iloc[i].close
                # order.send_order('BUY', 'OPEN', code=code, price=price, volume=1000)
#                 qa_order.send_order('SELL', 'CLOSE', code=code, price=price, volume=1000)

            elif cprice > hole_price * 1.2 and high_price / cprice > 1.06:
                data.iat[i, flag] = -1
                data.iat[i + 1, position_col] = 0
                data.iat[i + 1, hold_price_col] = 0
                position = 0
                print("sell : code=%s date=%s  price=%.2f" % (data.iloc[i].name[0], data.iloc[i].name[1], data.iloc[i].close))
                code = data.iloc[i].name[1]
                price = data.iloc[i].close
                # order.send_order('BUY', 'OPEN', code=code, price=price, volume=1000)
#                 qa_order.send_order('SELL', 'CLOSE', code=code, price=price, volume=1000)

            elif data.iat[i, sflag] > 0:
                data.iat[i, flag] = -1
                data.iat[i + 1, position_col] = 0
                data.iat[i + 1, hold_price_col] = 0
                position = 0
                print("sell : code=%s date=%s  price=%.2f" % (data.iloc[i].name[0], data.iloc[i].name[1], data.iloc[i].close))
                code = data.iloc[i].name[1]
                price = data.iloc[i].close
                # order.send_order('BUY', 'OPEN', code=code, price=price, volume=1000)
#                 qa_order.send_order('SELL', 'CLOSE', code=code, price=price, volume=1000)
            else:
                data.iat[i + 1, position_col] = data.iat[i, position_col]
                data.iat[i + 1, hold_price_col] = data.iat[i, hold_price_col]
        # 保持
        else:
            data.iat[i + 1, position_col] = data.iat[i, position_col]
            data.iat[i + 1, hold_price_col] = data.iat[i, hold_price_col]

    data['nav'] = (1+data.close.pct_change(1).fillna(0) * data.position).cumprod()
    data['nav1'] = data.close * data.position
    return data


In [ ]:
df=mongo.get_stock_day('600718')

In [ ]:
df.tail()

In [26]:
data1=buy_sell_fun(df1)

buy  : date=2018-04-16 00:00:00 code=600718 price=15.35
sell : code=2018-04-23 00:00:00 date=600718  price=14.97
buy  : date=2018-10-17 00:00:00 code=600718 price=9.61
sell : code=2018-10-25 00:00:00 date=600718  price=9.78
buy  : date=2019-04-01 00:00:00 code=600718 price=15.58
sell : code=2019-04-08 00:00:00 date=600718  price=15.02
buy  : date=2020-02-05 00:00:00 code=600718 price=11.96
sell : code=2020-02-11 00:00:00 date=600718  price=12.65
buy  : date=2020-07-14 00:00:00 code=600718 price=13.78
sell : code=2020-07-16 00:00:00 date=600718  price=12.28


In [ ]:
df1=tdx_base_func(df)

In [ ]:
df1.tail()

In [30]:
data1.loc['2018-04-10':]

,,amount,close,high,low,open,vol,bflg,sflg,ret,flag,position,hold_price,nav,nav1
date,code,,,,,,,,,,,,,,
2018-04-10,600718,4.809647e+08,14.52,15.00,14.30,14.96,330547.0,0,0,-0.034574,0,0,0,1.000000,0.00
2018-04-11,600718,3.096208e+08,14.54,14.82,14.45,14.62,211369.0,0,0,0.001377,0,0,0,1.000000,0.00
2018-04-12,600718,2.078606e+08,14.56,14.75,14.40,14.56,142528.0,0,0,0.001376,0,0,0,1.000000,0.00
2018-04-13,600718,3.983209e+08,14.71,15.28,14.70,15.20,266271.0,0,0,0.010302,0,0,0,1.000000,0.00
2018-04-16,600718,5.801121e+08,15.35,15.36,14.38,14.55,385949.0,1,0,0.043508,1,1,14,1.043508,15.35
2018-04-17,600718,5.744495e+08,14.63,15.65,14.38,15.35,382150.0,0,0,-0.046906,0,1,14,0.994562,14.63
2018-04-18,600718,6.655035e+08,15.60,15.84,14.26,14.66,441657.0,0,0,0.066302,0,1,14,1.060503,15.60
2018-04-19,600718,8.856690e+08,15.74,16.38,15.27,15.46,559552.0,0,0,0.008974,0,1,14,1.070020,15.74
2018-04-20,600718,7.582362e+08,15.73,16.22,15.45,15.50,475091.0,0,0,-0.000635,0,1,14,1.069341,15.73


In [92]:
a = np.array([10,11,13,15,12,7,14])
ap = np.array([1,1,1,1,1,0,0])
b = np.array([1.2,1.1,1.8,1.5,1.2,0.7,1.4])
# a = np.array([[10,11,13,15,12,7,14],[10,11,18,15,12,7,14]])

In [94]:
dfn=pd.Series(a)
dfb=pd.Series(b)
df=pd.DataFrame()
df['a']=pd.Series(a)
df['ap']=pd.Series(ap)
df

,a,ap
0,10,1
1,11,1
2,13,1
3,15,1
4,12,1
5,7,0
6,14,0


In [86]:
bc1=(1+dfn.pct_change(1).fillna(0)).cumprod()
bc2=(1+dfb.pct_change(1).fillna(0)).cumprod()

In [88]:
bc2

0    1.000000
1    0.916667
2    1.500000
3    1.250000
4    1.000000
5    0.583333
6    1.166667
dtype: float64

In [90]:
result01=bc2
round(float(max([(result01.iloc[idx] - result01.iloc[idx::].min()) / result01.iloc[idx] for idx in range(len(result01))])), 2)

0.61

In [54]:
bc

0    1.0
1    1.1
2    1.3
3    1.5
4    1.2
5    0.7
6    1.4
dtype: float64